In [1]:
# if the files are on Anvil / Terra workspace 
# please run it from the workspace analysis to avoid bucket access permission issues
# or use the gcloud login auth interface to authenticate yourself

In [2]:
!pwd

/Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out


installations:

1. pip install synapseclient

In [3]:
import os
import pandas as pd
# CHANGE
# gs_path_for_the_encode_pseudobulk_pipeline_table = "ENCODE_PSEUDOBULK_TABLE_INPUT"
# local_folder = os.getcwd() #"/home/jupyter/eila" #os.getcwd()
# os.makedirs(local_folder, exist_ok=True)
full_cell_types_annotation_file_path = "POINT TO YOUR FILE"
local_clusters_fld = os.path.join(os.getcwd(),"clusters")
os.makedirs(local_clusters_fld, exist_ok=True)
local_path_to_download = os.path.join(os.getcwd(),"fragment_files")

In [4]:
def remove_file(filename):
  print("remove_file method: {}".format(filename))
  if os.path.exists(filename):
    os.remove(filename)

In [5]:
# %run synapse_utils_from_analysis.ipynb
# %run tsv_files_utils_from_analysis.ipynb

TODO:
- make sure that your fragments file are in this directory

In [10]:
# load the list of the fragment
local_fragment_files_path = os.path.join(os.getcwd(),"fragment_files","*","*")
print("local_fragment_files_path")
local_fragment_files = !ls $local_fragment_files_path
local_fragment_files = list(local_fragment_files)
local_fragment_files[4]

local_fragment_files_path


'/Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out/fragment_files/syn52118179_syn52119935_ENCSR023FME.atac.filter.fragments.hg38.tsv.gz/ENCSR023FME.atac.filter.fragments.hg38.tsv.gz'

- check that the files in the fragment files has matching synapse data

TODO:
- make sure that you received the atac dataset is from this split

In [12]:
local_fragment_files[2].split("_")[-1].split(".")[0]

'ENCSR845QFX'

In [13]:
local_files_atac_dataset = [x.split("_")[-1].split(".")[0] for x in local_fragment_files]
local_files_atac_dataset[0:10]

['ENCSR411MLR',
 'ENCSR052DKH',
 'ENCSR845QFX',
 'ENCSR660NEE',
 'ENCSR023FME',
 'ENCSR453TVZ',
 'ENCSR868ZEI',
 'ENCSR474TGL',
 'ENCSR538FPK',
 'ENCSR181XXQ']

In [ ]:
# %run tsv_files_utils_from_analysis.ipynb
# %run synapse_utils_from_analysis.ipynb

# TODO: for others. will the data come from synpase as well?
#local_clusters_fld
#  I need to hvae one full_cell_types_annotation_file_path
for local_fragment_file in local_fragment_files[2:]:
    
    print("!!!!!local_fragment_file is {}".format(local_fragment_file))
    file_atac_dataset_id = local_fragment_file.split("_")[-1].split(".")[0]
    print("file_atac_dataset_id is {}".format(file_atac_dataset_id))
 
    # This full_cell_types_annotation_file_path was given as an input
    df_cell_types_for_atac_dataset = read_tsv_gz_to_dataframe_skipping_comments_and_empty_lines(full_cell_types_annotation_file_path)
    cell_type_id_names_for_atac_dataset = list(set(df_cell_types_for_atac_dataset['cell_type_id'].to_list()))
    print("number of cell_type_id_names_for_atac_dataset is {}".format(len(cell_type_id_names_for_atac_dataset)))
    # List of output text files with corresponding names
    # [("output1.txt", "file_1"), ("output2.txt", "file_2"), ("output3.txt", "file_3")]
    output_tagAlign_files_with_names = [(os.path.join(local_clusters_fld,file_atac_dataset_id,"tagAlign_{}_{}.tsv".format(file_atac_dataset_id,cell_type_name_id)),
                                        cell_type_name_id) for cell_type_name_id in cell_type_id_names_for_atac_dataset]
    
    # print("!!!output_tagAlign_files_with_names {}".format(output_tagAlign_files_with_names))
    print("open local_fragment_file {}".format(local_fragment_file))
    with gzip.open(local_fragment_file, "rt") as infile:
        # Dictionary to store the output file handles with names
        output_handles = {}
        missing_bc = 0
        # Open the output files and store their handles in the list
        for tag_file_path, tag_file_cell_type_name in output_tagAlign_files_with_names:
            print("tag_file_path is {}".format(tag_file_path))
            # print("os.path.dirname(tag_file_path) is {}".format(os.path.dirname(tag_file_path)))
            os.makedirs(os.path.dirname(tag_file_path), exist_ok=True)
            output_handles[tag_file_cell_type_name] = open(tag_file_path, "w")
            num_of_lines_written=0
            for line_number, line in enumerate(infile, start=1):
                # # debug
                # if line_number > 1000:
                #     continue 

                out_list = split_fragment_line_string(line)
                bc = out_list[-1]
#             Austin output: chrom, start, end, bc, rem = line.rstrip('\n').split('\t', 5)
                out_line_to_print = f"{out_list[0]}\t{out_list[1]}\t{out_list[2]}\t{bc}\t{out_list[-2]}\n"
#             chr1	10007	10175	ENCSR023FME#ENCSR023FME_GAAGGTTCAAAGTGTCAGTCAA	1
                num_of_lines_written +=1
                returnTagAlign = convert_fragment_line_to_tagAlign(out_line_to_print)
                # write to the relevant cell type file
                bc_exists_in_cell_type_atac_dataset = df_cell_types_for_atac_dataset[df_cell_types_for_atac_dataset['cell_id']==bc]
                # print("len(bc_exists_in_cell_type_atac_dataset) is {} for bc {}".format(len(bc_exists_in_cell_type_atac_dataset),bc))
                if len(bc_exists_in_cell_type_atac_dataset) ==1:
                    cell_name_for_line = df_cell_types_for_atac_dataset.loc[df_cell_types_for_atac_dataset['cell_id'] == bc, 'cell_type_name'].iloc[0]
                    # print("cell_name_for_line is {}".format(cell_name_for_line))
                    output_handles[tag_file_cell_type_name].write(returnTagAlign)
                else:
                    missing_bc+=1
                    
        print("finished clustering local_fragment_file {} by cell type. for types {}".format(local_fragment_file, cell_type_id_names_for_atac_dataset))
        print("total missing bc are {}".format(missing_bc))

Welcome, eila!

!!!!!local_fragment_file is /Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out/fragment_files/syn52118179_syn52119922_ENCSR411MLR.atac.filter.fragments.hg38.tsv.gz/ENCSR411MLR.atac.filter.fragments.hg38.tsv.gz
file_atac_dataset_id is ENCSR411MLR
syn_folder_name_for_cell_types in synpase is EAID_000014
get_file_names parameters are: folder_id syn34271785 and current_depth 0
current_depth is 0 
folder_entities are <generator object Synapse.getChildren at 0x294678820> 
get_file_names parameters are: folder_id syn46645618 and current_depth 1
current_depth is 1 
folder_entities are <generator object Synapse.getChildren at 0x294678280> 
get_file_names parameters are: folder_id syn46645622 and current_depth 2
folder_entities are <generator object Synapse.getChildren at 0x294678ca0>
syn_file_id_for_atac_dataset_id is syn46645631
cell_types_annotation_file_synHandle is File: cell_types.tsv.gz (syn